# Домашнее задание к лекции «Улучшение качества модели»

### Цель:
Применить на практике алгоритмы по автоматической оптимизации параметров моделей машинного обучения.

### Описание задания:  
В домашнем задании нужно решить задачу классификации наличия болезни сердца у пациентов наиболее эффективно. Данные для обучения моделей необходимо загрузить самостоятельно с сайта. Целевая переменная – наличие болезни сердца (HeartDisease). Она принимает значения 0 или 1 в зависимости от отсутствия или наличия болезни соответственно. Подробное описание признаков можно прочесть в описании датасета на сайте. Для выполнения работы не обязательно вникать в медицинские показатели.

### Этапы работы:

1. Получите данные и загрузите их в рабочую среду. (Jupyter Notebook или другую)  
2. Подготовьте датасет к обучению моделей:  
    a) Категориальные переменные переведите в цифровые значения. Можно использовать pd.get_dummies, preprocessing.LabelEncoder. Старайтесь не использовать для этой задачи циклы.
3. Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.
4. Обучите модель логистической регрессии с параметрами по умолчанию.
5. Подсчитайте основные метрики модели. Используйте следующие метрики и функцию: cross_validate(…, cv=10, scoring=[‘accuracy’,‘recall’,‘precision’,‘f1’])
6. Оптимизируйте 3-4 параметра модели:  
    a) Используйте GridSearchCV.  
    b) Используйте RandomizedSearchCV.  
    c) *Добавьте в п. 6b 2-5 моделей классификации и вариации их параметров.  
    d) Повторите п. 5 после каждого итогового изменения параметров.  
7. Сформулируйте выводы по проделанной работе:  
    a) Сравните метрики построенных моделей.  
    b) *Сравните с полученными результатами в домашнем задании по теме «Ансамблирование».  


**Для получения зачета по этому домашнему заданию минимально необходимо:**
- обучить одну модель классификации;
- оптимизировать параметры, используя метод из п. 6a;
- вывести значения метрик.

**Результат:**
- Получены знания по оптимизации параметров.
- Форма выполнения:
- ссылка на Jupyter Notebook, загруженный на GitHub;
- ссылка на Google Colab;
- файл с расширением .ipynb.

**Инструменты:**
- Jupyter Notebook/Google Colab;
- GitHub;
- сайт с данными для обучения моделей.

Срок выполнения: дедлайн приема решений на проверку

**Рекомендации к выполнению:**

- Текст оформляйте в отдельной ячейке Jupyter Notebook/Google Colab в формате markdown.
- У графиков должен быть заголовок, подписи осей, легенда (опционально). Делайте графики бОльшего размера, чем стандартный вывод, чтобы увеличить читабельность.
- Убедитесь, что по ссылкам есть доступ на чтение/просмотр.
- Убедитесь, что все ячейки в работе выполнены и можно увидеть их вывод без повторного запуска.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline


### 1. Получите данные и загрузите их в рабочую среду. (Jupyter Notebook или другую)

In [2]:
data = pd.read_csv('heart.csv')

data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


### 2. Подготовьте датасет к обучению моделей:
a) Категориальные переменные переведите в цифровые значения. Можно использовать pd.get_dummies, preprocessing.LabelEncoder.

In [3]:
data['ChestPainType'].value_counts()

ASY    496
NAP    203
ATA    173
TA      46
Name: ChestPainType, dtype: int64

In [4]:
data['RestingECG'].value_counts()

Normal    552
LVH       188
ST        178
Name: RestingECG, dtype: int64

In [5]:
data['ExerciseAngina'].value_counts()

N    547
Y    371
Name: ExerciseAngina, dtype: int64

In [6]:
data['ST_Slope'].value_counts()

Flat    460
Up      395
Down     63
Name: ST_Slope, dtype: int64

In [7]:
data['Sex'].value_counts()

M    725
F    193
Name: Sex, dtype: int64

In [3]:
categorical_features = ['ChestPainType', 'RestingECG',  'ST_Slope']
binary_features = ['Sex', 'ExerciseAngina']

data = pd.get_dummies(data, columns=categorical_features) 
data = pd.get_dummies(data, columns=binary_features, drop_first=True) # для бинарных признаков drop_first=True чтобы удалить один из столбов


data.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up,Sex_M,ExerciseAngina_Y
0,40,140,289,0,172,0.0,0,0,1,0,0,0,1,0,0,0,1,1,0
1,49,160,180,0,156,1.0,1,0,0,1,0,0,1,0,0,1,0,0,0
2,37,130,283,0,98,0.0,0,0,1,0,0,0,0,1,0,0,1,1,0
3,48,138,214,0,108,1.5,1,1,0,0,0,0,1,0,0,1,0,0,1
4,54,150,195,0,122,0.0,0,0,0,1,0,0,1,0,0,0,1,1,0


### 3.Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.

In [4]:
x = data.drop(columns='HeartDisease')
x.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up,Sex_M,ExerciseAngina_Y
0,40,140,289,0,172,0.0,0,1,0,0,0,1,0,0,0,1,1,0
1,49,160,180,0,156,1.0,0,0,1,0,0,1,0,0,1,0,0,0
2,37,130,283,0,98,0.0,0,1,0,0,0,0,1,0,0,1,1,0
3,48,138,214,0,108,1.5,1,0,0,0,0,1,0,0,1,0,0,1
4,54,150,195,0,122,0.0,0,0,1,0,0,1,0,0,0,1,1,0


In [5]:
y = data['HeartDisease']

y.head()

0    0
1    1
2    0
3    1
4    0
Name: HeartDisease, dtype: int64

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

### 4.Обучите модель логистической регрессии с параметрами по умолчанию.

In [10]:
from sklearn.linear_model import LogisticRegression


model_1 = LogisticRegression(max_iter=2000)

model_1.fit(x_train, y_train)

LogisticRegression(max_iter=2000)

In [11]:
model_1_pred_y = model_1.predict(x_test)

In [12]:
from sklearn.metrics import f1_score

model_1_score = f1_score(y_test, model_1_pred_y)
print(model_1_score)

0.8695652173913043


### 5. Подсчитайте основные метрики модели. Используйте следующие метрики и функцию: cross_validate(…, cv=10, scoring=[‘accuracy’,‘recall’,‘precision’,‘f1’])

In [13]:
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import cross_validate


model_lr = LogisticRegression(max_iter=2000)
scores_lr_cross_val = cross_validate(model_lr, x, y, cv=10, scoring = ['accuracy','recall','precision','f1'])

print(f"Accuracy score: {scores_lr_cross_val['test_accuracy'].mean()}")
print(f"Recall score: {scores_lr_cross_val['test_recall'].mean()}")
print(f"Precision score: {scores_lr_cross_val['test_precision'].mean()}")
print(f"F1 score: {scores_lr_cross_val['test_f1'].mean()}")


Accuracy score: 0.8516364070711898
Recall score: 0.8714901960784314
Precision score: 0.8665853755321589
F1 score: 0.8649052177666536


### 6.Оптимизируйте 3-4 параметра модели:
a) Используйте GridSearchCV.


In [14]:
from sklearn.model_selection import GridSearchCV

model_lr = LogisticRegression()

# Словарь с параметрами
params_lr = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l1', 'l2'],
    'max_iter': list(range(1000,2000,200)),
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

grid_lr = GridSearchCV(model_lr, params_lr, cv=10, scoring='f1')


In [42]:
grid_lr.fit(x,y)

In [16]:
print(grid_lr.best_score_)

print(grid_lr.best_params_)

print(grid_lr.best_estimator_)

0.8671855401366451
{'C': 1, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'liblinear'}
LogisticRegression(C=1, max_iter=1000, solver='liblinear')


In [17]:
grid_best_estimator = grid_lr.best_estimator_

grid_best_estimator.fit(x_train, y_train)

LogisticRegression(C=1, max_iter=1000, solver='liblinear')

In [18]:
grid_lr_pred_y = grid_best_estimator.predict(x_test)

In [19]:
grid_lr_score = f1_score(y_test, grid_lr_pred_y)
print(grid_lr_score)

0.8695652173913043


b) Используйте RandomizedSearchCV.

In [20]:
from sklearn.model_selection import RandomizedSearchCV

model_lr = LogisticRegression()

# Словарь с параметрами
params_lr = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l1', 'l2'],
    'max_iter': list(range(100,2000,200))
    #'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

random_lr = RandomizedSearchCV(model_lr, params_lr, cv=10, scoring='f1')


In [43]:
random_lr.fit(x,y)

In [22]:
print(random_lr.best_score_)

print(random_lr.best_params_)

print(random_lr.best_estimator_)

0.8651649008816339
{'penalty': 'l2', 'max_iter': 1500, 'C': 10}
LogisticRegression(C=10, max_iter=1500)


In [23]:
random_best_estimator = random_lr.best_estimator_

random_best_estimator.fit(x_train, y_train)

random_lr_pred_y = random_best_estimator.predict(x_test)

In [24]:
random_lr_score = f1_score(y_test, random_lr_pred_y)
print(random_lr_score)

0.8695652173913043


c) Добавьте в п. 6b 2-5 моделей классификации и вариации их параметров.


In [25]:
from sklearn.linear_model import Ridge
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import uniform


In [26]:
models = [
   # {'name':'NB',"model":BernoulliNB(), 'params':{'alpha': uniform(loc=0,scale=-4)}},
   # {'name':'R',"model":Ridge(), 'params':{'alpha':uniform(loc=0,scale=-4), 'solver':['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}},
   # {'name':'SVC',"model":SVC(), 'params':{'kernel':['linear', 'poly', 'rbf', 'sigmoid'], 'gamma':['scale', 'auto']}},
    {'name':'RF',"model":RandomForestClassifier(), 'params':{'n_estimators':[10,25,50,100,150,200], 'criterion':['gini', 'entropy'], 'max_depth':[3,5,7,9,11]}},
    {'name':'KN',"model":KNeighborsClassifier(), 'params':{'n_neighbors':list(range(1,30)), 'weights':['uniform', 'distance'], 'p':[1,2,3]}},
    {'name':'DT',"model":DecisionTreeClassifier(), 'params':{'criterion':['gini', 'entropy'], 'max_depth':[3,5,7,9,11]}}
]




In [27]:
import warnings
warnings.filterwarnings("ignore")

res = []

for v in models:
    random_search = RandomizedSearchCV(v['model'], v['params'], cv=10, scoring='f1').fit(x_train, y_train)
    res.append((v['name'], random_search.best_params_, random_search.best_score_,  random_search.best_estimator_))
   


d) Повторите п. 5 после каждого итогового изменения параметров.

In [29]:
res

[('RF',
  {'n_estimators': 100, 'max_depth': 7, 'criterion': 'gini'},
  0.8871741609117043,
  RandomForestClassifier(max_depth=7)),
 ('KN',
  {'weights': 'distance', 'p': 1, 'n_neighbors': 17},
  0.78393181100532,
  KNeighborsClassifier(n_neighbors=17, p=1, weights='distance')),
 ('DT',
  {'max_depth': 3, 'criterion': 'entropy'},
  0.8607486315719232,
  DecisionTreeClassifier(criterion='entropy', max_depth=3))]

### 7. Сформулируйте выводы по проделанной работе:
a) Сравните метрики построенных моделей.  

In [41]:
print(f"LogisticRegression без подбора параметров F1 score: {model_1_score}")

print(f"LogisticRegression средий F1 score после кросс валидации: {scores_lr_cross_val['test_f1'].mean()}")

print(f"LogisticRegression с параметрами подобранными сеткой F1 score: {grid_lr.best_score_}")

print(f"LogisticRegression с параметрами подобранными случайно F1 score: {random_lr.best_score_}")

print(f"RandomForestClassifier с параметрами подобранными случайно F1 score: {res[0][2]}")

print(f"KNeighborsClassifier с параметрами подобранными случайно F1 score: {res[1][2]}")

print(f"DecisionTreeClassifier с параметрами подобранными случайно F1 score: {res[2][2]}")

# Как можем видеть логистическая регрессия дала хороший результат без особого подбора параметров
# Но случайный лес дал более хороший результат после подбора параметров


LogisticRegression без подбора параметров F1 score: 0.8695652173913043
LogisticRegression средий F1 score после кросс валидации: 0.8649052177666536
LogisticRegression с параметрами подобранными сеткой F1 score: 0.8671855401366451
LogisticRegression с параметрами подобранными случайно F1 score: 0.8651649008816339
RandomForestClassifier с параметрами подобранными случайно F1 score: 0.8871741609117043
KNeighborsClassifier с параметрами подобранными случайно F1 score: 0.78393181100532
DecisionTreeClassifier с параметрами подобранными случайно F1 score: 0.8607486315719232


b) Сравните с полученными результатами в домашнем задании по теме «Ансамблирование».

In [ ]:
# В прошлом ДЗ самый высокий результат был у случайного леса со следующим резльтутом:
# F1 score on test data with Random Forest 0.8985507246376813

# Почему-то тогда получился более хороший результат без подбора параметров. Возможно это произошло случайно из-за того как разбились
# данные на тренировчоные и тестовые 